In [ ]:
import sys
import os.path
#import pikepdf
#import urllib.request
import pandas as pd
import json
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches


In [ ]:
#data = pd.read_csv("data/meta.csv")#header=None
fusion = pd.read_json("data/fusedcontent.json", orient = 'table').replace("", np.nan)
data = fusion[(fusion['Identifier'] != '')]

fusion

### Hier folgen statistische Auswertungen aus dem gewonnenen Daten der ersten Phase

In [ ]:
keyword_statistics=data.groupby(['Keywords'])["Keywords"]\
                                       .count() \
                                       .reset_index(name='count') \
                                       .sort_values(['count'], ascending=False, ignore_index=True)

keyword_statistics.to_csv("data/metaauswertungkeywords.csv", index=False)
keyword_statistics

In [ ]:
data["Keywords"].count()

In [ ]:
error_statistics = data.groupby(['Error'])["Error"]\
                                       .count() \
                                       .reset_index(name='count') \
                                       .sort_values(['count'], ascending=False, ignore_index=True)

error_statistics

In [ ]:
error_list = error_statistics['count'].tolist()
error_list.insert(0, len(data)-error_statistics['count'].sum())

error_name = error_statistics['Error'].tolist()
error_name.insert(0, 'No Error')

colors = ['green', 'red', 'blue', 'black', 'orange', 'violet']

y_offset=0
for error in range(len(error_list)):
    plt.barh(y= 0, 
             width = error_list[error], 
             left = y_offset, 
             color = colors[error],
             label = error_name[error])
    
    y_offset = y_offset + error_list[error]
plt.axis('off')
plt.legend(loc='upper left')

In [ ]:
index = np.arange(len(error_statistics))
plt.bar_label(plt.bar(x= index, height=error_statistics['count']), label=error_statistics['Error'])

In [ ]:
author_statistics = data.groupby(['Author'])["Author"]\
                                       .count() \
                                       .reset_index(name='count') \
                                       .sort_values(['count'], ascending=False, ignore_index=True)
                                       
                                       
#author_statistics.to_csv("data/metaauswertungauthor.csv", index=False)
author_statistics.head()

In [ ]:
cumsum_statistics = author_statistics.cumsum()
cumsum_statistics["ratio"] = cumsum_statistics["count"] / cumsum_statistics["count"].max()

ax = cumsum_statistics.ratio.plot()
ax.set_title("Contributor cumsum count of OER Materials")
ax.set_xlabel("Contributor count")
ax.set_ylabel("Ratio")

plt.plot([34],[0.5], "+r")
plt.plot([169],[0.8], "+r")

In [ ]:

timedata = pd.DataFrame(data['Datum der Erstellung'])
timedata = timedata.dropna()
timelist=list()

for index, row in timedata.iterrows():
    #print(row['Datum der Erstellung'])
    timelist.append(datetime.strptime(row['Datum der Erstellung'],"%Y/%m/%d"))
timedata = timedata.assign(datetime=timelist)
#df1 = df1.assign(e=pd.Series(np.random.randn(sLength)).values)
    #timelist.append(row.strftime(%x))
timedata.drop(columns='Datum der Erstellung', inplace=True)   
    
dailytime_statistics = timedata.groupby(['datetime'])["datetime"]\
                                       .count() \
                                       .reset_index(name='count') \
                                       .sort_values(['datetime'], ascending=False, ignore_index=True)

#time_statistics.to_csv("temp/metaauswertungtime.csv", index=False)
dailytime_statistics

In [ ]:
dailytime_statistics.set_index(['datetime'],inplace=True)
dailytime_statistics.plot()

In [ ]:
weeklytime_statistics = timedata.resample('W', on='datetime').count() #hier Buchstaben verändern für andere Zeitspannen

weeklytime_statistics.rename(columns={list(weeklytime_statistics)[0]:'count'}, inplace=True)
weeklytime_statistics.plot()

In [ ]:
pagedata = data[('Seitenzahl' )].replace('', np.nan).dropna()
page_statistics = data.groupby(['Seitenzahl'])["Seitenzahl"]\
                                       .count() \
                                       .reset_index(name='count') \
                                       .sort_values(['Seitenzahl'], ascending=True, ignore_index=True)
page_statistics

In [ ]:
page_statistics.set_index(['Seitenzahl'],inplace=True)
page_statistics.plot()

In [ ]:
format_statistics = data.groupby(['Format'])["Format"]\
                                       .count() \
                                       .reset_index(name='count') \
                                       .sort_values(['count'], ascending=False, ignore_index=True)
format_statistics

In [ ]:
format_statistics.plot(kind='bar', x='Format')

In [ ]:
#Dateigröße
sizedata = pd.DataFrame(data['Dateigröße'])
sizedata.dropna(inplace=True)

#sizelist = list()

#for index, row in sizedata.iterrows():
#    sizesplitter = row['Dateigröße'].split(" ", 1)
#    if sizesplitter[1] == "GB":
#        sizelist.append(float(sizesplitter[0]) * 1000000000)
#    elif sizesplitter[1] == "MB":
#        sizelist.append(float(sizesplitter[0]) * 1000000)
#    elif sizesplitter[1] == "kB":
#        sizelist.append(float(sizesplitter[0]) * 1000)
#    else:
#        sizelist.append(float(sizesplitter[0]))

#sizelist
#sizedata = sizedata.assign(size=sizelist)


#df1 = df1.assign(e=pd.Series(np.random.randn(sLength)).values)
    #timelist.append(row.strftime(%x))

size_statistics = sizedata.groupby(['Dateigröße'])["Dateigröße"]\
                                       .count() \
                                       .reset_index(name='count') \
                                       .sort_values(['Dateigröße'], ascending=True, ignore_index=True)
size_statistics

In [ ]:
size_statistics.set_index(['Dateigröße'],inplace=True)
size_statistics.plot()

### Hier werden Daten aus content.json und allen Phasen ausgewertet

In [ ]:
#delete Errors
fusion = fusion[(fusion['Error'].isnull())]

maxsize = len(fusion)
licenseprev = len(fusion[((fusion['license'].notna()) & (fusion['license'] != ''))])
titleprev = len(fusion[((fusion['title'].notna()) & (fusion['title'] != ''))])
titlenow = len(fusion[((fusion['title'].notna()) & (fusion['title'] != '')) | ((fusion['Titel'].notna()) & (fusion['Titel'] != ''))])
creatorprev = len(fusion[((fusion['creator'].notna()) & (fusion['creator'] != ''))])
creatornow =len(fusion[((fusion['creator'].notna()) & (fusion['creator'] != '')) | ((fusion['Author'].notna()) & (fusion['Author'] != ''))])
pagesprev = len(fusion[((fusion['pages'].notna()) & (fusion['pages'] != ''))])
pagesnow = len(fusion[((fusion['pages'].notna()) & (fusion['pages'] != '')) | ((fusion['Seitenzahl'].notna()) & (fusion['Seitenzahl'] != ''))])
languageprev = len(fusion[((fusion['language'].notna()) & (fusion['language'] != ''))])
keywordsprev = 0 #TODO what from content masps to keywords
keywordsnow = len(fusion[((fusion['Keywords'].notna()) & (fusion['Keywords'] != ''))])
#keywords

columns = ('License', 'Title', 'Creator', 'Pages', 'Language', 'Keywords')
ind =[0.2, 0.65, 1.1, 1.55, 2, 2.45]

In [ ]:
#Graph w/o meta.csv

plt.bar(x = ind, 
        height = [licenseprev/maxsize, titleprev/maxsize, creatorprev/maxsize, pagesprev/maxsize, languageprev/maxsize, keywordsprev/maxsize], 
        color ='lightblue',
        width = 0.4,
        tick_label = columns)

In [ ]:
#Graph with meta.csv
plt.bar(x = ind, 
        height = [licenseprev/maxsize, titlenow/maxsize, creatornow/maxsize, pagesnow/maxsize, languageprev/maxsize, keywordsnow/maxsize], 
        color ='lightblue',
        width = 0.4,
        tick_label = columns)

In [ ]:
plt.bar(x = ind, 
        height = [licenseprev/maxsize, titlenow/maxsize, creatornow/maxsize, pagesnow/maxsize, languageprev/maxsize, keywordsnow/maxsize], 
        color ='lightblue',
        width = 0.4,
        tick_label = columns)
plt.bar(x = ind, 
        height = [licenseprev/maxsize, titleprev/maxsize, creatorprev/maxsize, pagesprev/maxsize, languageprev/maxsize, keywordsprev/maxsize], 
        color ='blue',
        width = 0.4,
        tick_label = columns)

content_label = mpatches.Patch(color='blue', label='content.json')
fusion_label = mpatches.Patch(color='lightblue', label='fused content')
plt.legend(handles=[content_label, fusion_label])


In [ ]:
titlecompare = fusion[((fusion['title'].notna()) & (fusion['Titel'].notna()))]
len(titlecompare)

In [ ]:
titlesame = titlecompare[(titlecompare['title'] == titlecompare['Titel'])]
len(titlesame)

In [ ]:
#titlesame.to_csv("data/metaauswertungtitelvergleich.csv", index=False)